In [1]:
import pandas as pd

df = pd.read_csv('train.tsv', sep='\t')

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

df = df [:120]
vectorizer = CountVectorizer(binary=True)
text_data = df["Phrase"]
text_data = vectorizer.fit_transform(text_data)

import torch

text_tensor = torch.from_numpy(text_data.todense()).float()

#ajout du cas où on ne connait pas le mot
voca = vectorizer.get_feature_names() + list(['<unk>'])
#ajout d'un zéro à la fin de chaque tensor
unk = torch.zeros([text_tensor.shape[0],1], dtype=torch.float)
text_tensor = torch.cat((text_tensor, unk), 1).unsqueeze(1)

label_one_hot_vector = torch.tensor(pd.get_dummies(df["Sentiment"]).values)
label_tensor = torch.tensor(df["Sentiment"])

label_one_hot_vector

tensor([[0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, embedding_size)

        self.rnn = nn.RNN(input_size = input_size, hidden_size = hidden_size, batch_first = True)
        self.h2o = nn.Linear(hidden_size, output_size)       
        
    def forward(self, input, hidden):
        
        hidden = hidden.to(device)
        output, hidden = self.rnn(input, hidden)
        output = self.h2o(hidden)
        
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, dtype=torch.float)


In [30]:
input_dim = len(voca)
embedding_size = 100
hidden_dim = 100
output_dim = 5

rnn = RNN(input_dim, embedding_size, hidden_dim, output_dim)

import torch.optim as optim

learning_rate = 0.005

optimizer = optim.SGD(rnn.parameters(), lr=1e-3)

def train(category_tensor, category_hov, text_tensor, num_epoch, batch_size):
    
    category_tensor = category_tensor.to(device)
    text_tensor = text_tensor.to(device)
    
    hidden = rnn.initHidden(batch_size)   
    criterion = nn.CrossEntropyLoss()    
    rnn.zero_grad()
    
    
    
    for epoch in range(num_epoch):
        epoch_loss = 0
        epoch_acc = 0
        i = 0
        while i < text_tensor.size(0):
            i += batch_size
            input = text_tensor[i-batch_size:i]
            target = category_tensor[i-batch_size:i]
            target_hov = category_hov[i-batch_size:i]
            
            output, hidden = rnn(input, hidden)

            loss = criterion(output.squeeze(0), target)
            loss.backward(retain_graph=True)
            optimizer.step() 
            
            epoch_loss += loss.item()
            
            print(epoch_loss)
    print('Fini !')

In [31]:
current_loss = 0
batch_size = 2

train(label_tensor, label_one_hot_vector, text_tensor, 1, batch_size)

# current_loss += loss

<class 'torch.Tensor'>


RuntimeError: Expected object of scalar type Long but got scalar type Byte for argument #2 'other'